# Raw data length binning

The purpose of this notebook is to design a strategy to load text fragment data with perplexity ratio score feature generated by the scoring algorithm, bin it by text fragment length and get it ready for input into feature engineering. The plan is to handle data as pandas dataframes and store it using hdf5. The text fragments will be put into overlapping length bins so that feature engineering and classifier training can be conducted separately for different length regimes. This general strategy is based on early observations of classifier performance on short fragments, long fragments and un-binned fragments.

## 1. Run set-up

In [1]:
# Change working directory to parent so we can import as we would from __main__.py
print(f'Working directory: ', end = '')
%cd ..
print()

import h5py
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn.preprocessing import LabelEncoder

import configuration as config

print(f'H5py: {h5py.__version__}')
print(f'Numpy: {np.__version__}')
print(f'Pandas: {pd.__version__}')
print(f'Sklearn: {sk.__version__}')

Working directory: /mnt/arkk/llm_detector/classifier

H5py: 3.11.0
Numpy: 1.24.4
Pandas: 2.0.3
Sklearn: 1.3.2


In [2]:
# The dataset we want to bin - omit the file extension, it will be 
# added appropriately for the input and output files
dataset_name = 'falcon-7b_scores_v2_10-1000_words'

# Construct input and output file paths
input_file = f'{config.HANS_DATA_PATH}/{dataset_name}.json'
output_file = f'{config.DATA_PATH}/{dataset_name}.h5'

# The bins we are planning on using
bins = {
    'combined': [0, np.inf],
    'bin_100': [1, 100],
    'bin_150': [51, 150],
    'bin_200': [101, 200],
    'bin_250': [151, 250],
    'bin_300': [201, 300],
    'bin_350': [251, 350],
    'bin_400': [301, 400],
    'bin_450': [351, 450],
    'bin_500': [401, 500],
    'bin_600': [451, 600]
}

## 2. Load and clean data

In [3]:
# Load the data
data_df = pd.read_json(input_file)

# Replace and remove string 'OOM' and 'NAN' values
data_df.replace('NAN', np.nan, inplace = True)
data_df.replace('OOM', np.nan, inplace = True)
data_df.dropna(inplace = True)

# Shuffle the deck, resetting the index
data_df = data_df.sample(frac = 1).reset_index(drop = True)

# Use the index to add a unique fragment id
data_df.reset_index(inplace = True)
data_df.rename({'index': 'Fragment ID'}, axis = 1, inplace = True)

# Enforce dtypes
data_df = data_df.astype({
    'Fragment ID': np.int64,
    'Source record num': np.int64,
    'Fragment length (words)': np.int64,
    'Fragment length (tokens)': np.int64,
    'Dataset': object, #pd.StringDtype(),
    'Source': object, #pd.StringDtype(),
    'Generator': object, #pd.StringDtype(),
    'String': object, #pd.StringDtype(),
    'Perplexity': np.float64,
    'Cross-perplexity': np.float64,
    'Perplexity ratio score': np.float64,
    'Reader time (seconds)': np.float64,
    'Writer time (seconds)': np.float64,
    'Reader peak memory (GB)': np.float64,
    'Writer peak memory (GB)': np.float64
})

data_df.head()

,Fragment ID,Source record num,Fragment length (words),Fragment length (tokens),Dataset,Source,Generator,String,Perplexity,Cross-perplexity,Perplexity ratio score,Reader time (seconds),Writer time (seconds),Reader peak memory (GB),Writer peak memory (GB)
0,0,9929,22,42,pubmed,synthetic,llama2-13b,ually. of the over 100 known malaria-causing p...,3.059,3.333984,0.917399,1.076827,1.164352,4.817057,4.810353
1,1,9216,134,180,pubmed,human,human,"the follow - up , such as aseptic loosening , ...",2.940,3.232422,0.909366,2.585715,2.813967,7.906423,7.798015
2,2,6913,181,237,cnn,synthetic,llama2-13b,earlier this week for the murders of his two s...,2.550,2.837891,0.898830,3.221827,3.518277,6.824884,6.757401
3,3,1643,215,273,cnn,human,human,Saturday that the Nigerian terrorist group Bok...,2.426,2.431641,0.997590,3.735403,4.183257,5.924875,5.921060
4,4,10016,92,145,pubmed,human,human,"active transmission of the pathogen , \n in sp...",3.447,3.324219,1.037015,2.500953,2.671648,7.706860,7.589395


In [4]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34196 entries, 0 to 34195
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Fragment ID               34196 non-null  int64  
 1   Source record num         34196 non-null  int64  
 2   Fragment length (words)   34196 non-null  int64  
 3   Fragment length (tokens)  34196 non-null  int64  
 4   Dataset                   34196 non-null  object 
 5   Source                    34196 non-null  object 
 6   Generator                 34196 non-null  object 
 7   String                    34196 non-null  object 
 8   Perplexity                34196 non-null  float64
 9   Cross-perplexity          34196 non-null  float64
 10  Perplexity ratio score    34196 non-null  float64
 11  Reader time (seconds)     34196 non-null  float64
 12  Writer time (seconds)     34196 non-null  float64
 13  Reader peak memory (GB)   34196 non-null  float64
 14  Writer

## 3. Generate hdf5 data structure

OK, let's start structuring the dataset. We want two top level groups, one for training data and one for reserved testing data. Inside those groups will live datasets for the bins. We will also use attributes to save some metadata etc.

In [5]:
# Prepare the hdf5 output - create or open for read/write
output = h5py.File(output_file, 'a')

# Create the top-level groups
_ = output.require_group('training')
_ = output.require_group('testing')

print(f'Top level groups: {(list(output.keys()))}')

# Next, we need to add a group for each fragment length bin,
# and one for the un-binned data
for group in output.keys():

    # Add the un-binned data group
    _ = output.require_group(f'{group}/combined')

    # Loop on the bins and add a group for each
    for bin in bins.keys():
        _ = output.require_group(f'{group}/{bin}')

# Finally, add the bins dictionary as metadata
output.attrs.update(bins)

print(f'\nBin attributes:')
for key, value in output.attrs.items():
    print(f' {key}: {value}')

output.close()

Top level groups: ['master', 'testing', 'training']

Bin attributes:
 bin_100: [  1 100]
 bin_150: [ 51 150]
 bin_200: [101 200]
 bin_250: [151 250]
 bin_300: [201 300]
 bin_350: [251 350]
 bin_400: [301 400]
 bin_450: [351 450]
 bin_500: [401 500]
 bin_600: [451 600]
 combined: [ 0. inf]


## 4. Populated hdf5 data structure

OK, the basic data structure is ready to go. Next thing to do is add data.

In [6]:
def make_labels(training_df, testing_df):
    '''Takes training and testing dataframes, gets and encode human/synthetic
    labels, encodes labels and returns. Note: this function leaves the 
    'Source' label column in the features dataframe for downstream use in 
    feature engineering.'''

    # Get the labels
    training_labels = training_df['Source']
    testing_labels = testing_df['Source']

    # Encode string class values as integers
    label_encoder = LabelEncoder()
    label_encoder = label_encoder.fit(training_labels)
    training_labels = pd.Series(label_encoder.transform(training_labels)).astype(np.int64)
    testing_labels = pd.Series(label_encoder.transform(testing_labels)).astype(np.int64)

    return training_labels, testing_labels

In [7]:
# Reopen out hdf5 file with pandas so we can work with dataframes
data_lake = pd.HDFStore(output_file)

# Add the raw data set at the top level as 'master' incase we want it later.
data_lake['master'] = data_df

# Next, get rid of un-trainable/unnecessary features
feature_drops = [
    'Fragment ID',
    'Source record num',
    'Dataset',
    'Generator',
    'Reader time (seconds)',
    'Writer time (seconds)',
    'Reader peak memory (GB)',
    'Writer peak memory (GB)'
]

data_df.drop(feature_drops, axis = 1, inplace = True)

# Split the data into training and testing
training_df = data_df.sample(frac = 0.7, random_state = 42)
testing_df = data_df.drop(training_df.index)

# Now do the same for the bins
for bin_id, bin_range in bins.items():

    # Pull the fragments for this bin
    bin_training_df = training_df[(training_df['Fragment length (words)'] >= bin_range[0]) & (training_df['Fragment length (words)'] <= bin_range[1])]
    bin_testing_df = testing_df[(testing_df['Fragment length (words)'] >= bin_range[0]) & (testing_df['Fragment length (words)'] <= bin_range[1])]

    # Fix the index
    bin_training_df.reset_index(inplace = True, drop = True)
    bin_testing_df.reset_index(inplace = True, drop = True)

    # Split un-binned data into features and labels
    bin_training_labels, bin_testing_labels = make_labels(bin_training_df, bin_testing_df)

    # Add the data to the data lake
    data_lake.put(f'training/{bin_id}/features', bin_training_df)
    data_lake.put(f'training/{bin_id}/labels', bin_training_labels)
    data_lake.put(f'testing/{bin_id}/features', bin_training_df)
    data_lake.put(f'testing/{bin_id}/labels', bin_training_labels)

data_lake.close()

## 5. Sanity check results

In [8]:
# Open the data lake to check out the result
data_lake = h5py.File(output_file, 'a')

# Print the result
for group in data_lake.keys():
    print(f'\n{group} contains:')

    for subgroup in data_lake[group]:

        if 'bin' in subgroup:
            print(f'  {subgroup} contains: ', end = '')

            for subsubgroup in data_lake[group][subgroup].keys():
                print(f'{subsubgroup} ', end = '')
            
            print()

        else:
            print(f' {subgroup}')

data_lake.close()


master contains:
 axis0
 axis1
 block0_items
 block0_values
 block1_items
 block1_values
 block2_items
 block2_values

testing contains:
  bin_100 contains: features labels 
  bin_150 contains: features labels 
  bin_200 contains: features labels 
  bin_250 contains: features labels 
  bin_300 contains: features labels 
  bin_350 contains: features labels 
  bin_400 contains: features labels 
  bin_450 contains: features labels 
  bin_500 contains: features labels 
  bin_600 contains: features labels 
  combined contains: features labels 

training contains:
  bin_100 contains: features labels 
  bin_150 contains: features labels 
  bin_200 contains: features labels 
  bin_250 contains: features labels 
  bin_300 contains: features labels 
  bin_350 contains: features labels 
  bin_400 contains: features labels 
  bin_450 contains: features labels 
  bin_500 contains: features labels 
  bin_600 contains: features labels 
  combined contains: features labels 


In [9]:
# Reopen out hdf5 file with pandas so we can work with dataframes
data_lake = pd.HDFStore(output_file)

print('Combined training features:\n')
print(data_lake['training/combined/features'].info())
print('\nCombined training labels:\n')
print(data_lake['training/combined/labels'].info())
print('\nBin 100 training features:\n')
print(data_lake['training/bin_100/features'].info())
print('\nBin 100 training labels:\n')
print(data_lake['training/bin_100/labels'].info())

data_lake.close()

Combined training features:

<class 'pandas.core.frame.DataFrame'>
Index: 23937 entries, 0 to 23936
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Fragment length (words)   23937 non-null  int64  
 1   Fragment length (tokens)  23937 non-null  int64  
 2   Source                    23937 non-null  object 
 3   String                    23937 non-null  object 
 4   Perplexity                23937 non-null  float64
 5   Cross-perplexity          23937 non-null  float64
 6   Perplexity ratio score    23937 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 1.5+ MB
None

Combined training labels:

<class 'pandas.core.series.Series'>
Index: 23937 entries, 0 to 23936
Series name: None
Non-Null Count  Dtype
--------------  -----
23937 non-null  int64
dtypes: int64(1)
memory usage: 374.0 KB
None

Bin 100 training features:

<class 'pandas.core.frame.DataFrame'>
Index: 8503 e

In [10]:
# Reopen out hdf5 file with pandas so we can work with dataframes
data_lake = pd.HDFStore(output_file)

for (path, subgroups, subkeys) in data_lake.walk():

    for subgroup in subgroups:

        print("GROUP: {}/{}".format(path, subgroup))

    for subkey in subkeys:

        key = "/".join([path, subkey])

        print("KEY: {}".format(key))

data_lake.close()

GROUP: /testing
GROUP: /training
KEY: /master
GROUP: /testing/bin_100
GROUP: /testing/bin_150
GROUP: /testing/bin_200
GROUP: /testing/bin_250
GROUP: /testing/bin_300
GROUP: /testing/bin_350
GROUP: /testing/bin_400
GROUP: /testing/bin_450
GROUP: /testing/bin_500
GROUP: /testing/bin_600
GROUP: /testing/combined
GROUP: /training/bin_100
GROUP: /training/bin_150
GROUP: /training/bin_200
GROUP: /training/bin_250
GROUP: /training/bin_300
GROUP: /training/bin_350
GROUP: /training/bin_400
GROUP: /training/bin_450
GROUP: /training/bin_500
GROUP: /training/bin_600
GROUP: /training/combined
KEY: /training/bin_100/features
KEY: /training/bin_100/labels
KEY: /training/bin_150/features
KEY: /training/bin_150/labels
KEY: /training/bin_200/features
KEY: /training/bin_200/labels
KEY: /training/bin_250/features
KEY: /training/bin_250/labels
KEY: /training/bin_300/features
KEY: /training/bin_300/labels
KEY: /training/bin_350/features
KEY: /training/bin_350/labels
KEY: /training/bin_400/features
KEY: /tra

OK, I think we are happy with this to start with. Let's move on and do some feature engineering in the bins.